In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import eli5
from eli5.sklearn import PermutationImportance


In [ ]:
#Importing DataSets
train_df=pd.read_csv('C:/Users/UF145GV/Documents/DL/testing/Healthcare Analytics/Train/Train.csv')
patient_df=pd.read_csv('C:/Users/UF145GV/Documents/DL/testing/Healthcare Analytics/Train/Patient_Profile.csv')
Health_camp_df=pd.read_csv('C:/Users/UF145GV/Documents/DL/testing/Healthcare Analytics/Train/Health_Camp_Detail.csv')
first_healthcamp_df=pd.read_csv('C:/Users/UF145GV/Documents/DL/testing/Healthcare Analytics/Train/First_Health_Camp_Attended.csv')
second_healthcamp_df=pd.read_csv('C:/Users/UF145GV/Documents/DL/testing/Healthcare Analytics/Train/Second_Health_Camp_Attended.csv')
third_healthcamp_df=pd.read_csv('C:/Users/UF145GV/Documents/DL/testing/Healthcare Analytics/Train/Third_Health_Camp_Attended.csv')

test_df=pd.read_csv('C:/Users/UF145GV/Documents/DL/testing/Healthcare Analytics/test_l0Auv8Q.csv')

In [ ]:
print(len(np.unique(Health_camp_df['Health_Camp_ID'])))

In [ ]:
#Merge First Health Camp Details
train_df=pd.merge(train_df,first_healthcamp_df, on=['Patient_ID','Health_Camp_ID'], how='left')
train_df=train_df.drop(['Unnamed: 4'], axis=1)
#Merge Second Health Camp Details
train_df=pd.merge(train_df,second_healthcamp_df, on=['Patient_ID', 'Health_Camp_ID'], how='left')
#Merge Third Health Camp Details
train_df=pd.merge(train_df,third_healthcamp_df, on=['Patient_ID', 'Health_Camp_ID'], how='left')


train_df.head(10)

In [ ]:
train_df.info()

In [ ]:
#Making Target Variable(favourable Outcome)

train_df['Outcome']=np.where((train_df['Health_Score'].isnull()==False) | (train_df['Health Score'].isnull()==False) | (train_df['Number_of_stall_visited'].isnull()==False),1,0)
train_df=train_df.drop(['Donation','Health_Score','Health Score','Number_of_stall_visited','Last_Stall_Visited_Number'], axis=1)

In [ ]:
train_df['Outcome'].value_counts()

In [ ]:
train_df.head(10)

In [ ]:
#Merging with Patient Details
train_df=pd.merge(train_df,patient_df, on='Patient_ID', how='left', validate='m:1')
#Merging With Health_Camp_Details
train_df=pd.merge(train_df,Health_camp_df, on='Health_Camp_ID', how='left', validate='m:1')

train_df.shape

In [ ]:
test_df=pd.merge(test_df,patient_df, on='Patient_ID', how='left', validate='m:1')

test_df=pd.merge(test_df,Health_camp_df, on='Health_Camp_ID', how='left', validate='m:1')

test_df.shape


In [ ]:
train_df=train_df[train_df['Registration_Date'].isnull()==False]
# train_df['Registration_Date'].fillna('')
train_df.info()

In [ ]:
combine_set=pd.concat([train_df, test_df], ignore_index=True)
combine_set.shape

In [ ]:
combine_set=combine_set[combine_set['Education_Score']!='101']

In [ ]:
#Transforming Category Variables
combine_set['City_Type'].fillna('J', inplace=True)
combine_set['Education_Score']=combine_set['Education_Score'].replace('None',0)
combine_set['Education_Score']=combine_set['Education_Score'].astype(float)
combine_set['Education_Score']=np.log1p(combine_set['Education_Score'])

combine_set['Age']=combine_set['Age'].replace('None',25)
combine_set['Age']=combine_set['Age'].astype(int)
combine_set['Age']=np.log1p(combine_set['Age'])

combine_set['Income']=combine_set['Income'].replace('None',7)
combine_set['Income']=combine_set['Income'].astype(int)

# from sklearn.preprocessing import LabelEncoder

# le=LabelEncoder()

# combine_set['City_Type']=le.fit_transform(combine_set['City_Type'])

# combine_set['Category2']=le.fit_transform(combine_set['Category2'])
fe=combine_set.groupby('City_Type').size()/len(combine_set)
combine_set['City_Type']=combine_set['City_Type'].map(fe)

fe=combine_set.groupby('Category2').size()/len(combine_set)
combine_set['Category2']=combine_set['Category2'].map(fe)

combine_set['Category1']=combine_set['Category1'].map({'First':1, 'Second':2, 'Third':3})


combine_set.head(10)

In [ ]:
combine_set['Var1']=np.log1p(combine_set['Var1'])

combine_set['Var2']=np.log1p(combine_set['Var2'])

combine_set['Var5']=np.log1p(combine_set['Var5'])

combine_set.info()

In [ ]:
#Obtaining New Features
combine_set['Registration_Date']=pd.to_datetime(combine_set['Registration_Date'], format='%d-%b-%y')
combine_set['Registration_Date']=combine_set['Registration_Date'].apply(lambda x: x.toordinal())
combine_set['Camp_Start_Date']=pd.to_datetime(combine_set['Camp_Start_Date'], format='%d-%b-%y')
combine_set['Camp_Start_Date']=combine_set['Camp_Start_Date'].apply(lambda x: x.toordinal())
combine_set['Camp_End_Date']=pd.to_datetime(combine_set['Camp_End_Date'], format='%d-%b-%y')
combine_set['Camp_End_Date']=combine_set['Camp_End_Date'].apply(lambda x: x.toordinal())
combine_set['First_Interaction']=pd.to_datetime(combine_set['First_Interaction'], format='%d-%b-%y')
combine_set['First_Interaction']=combine_set['First_Interaction'].apply(lambda x: x.toordinal())

combine_set['No_of_camp_days']=combine_set['Camp_End_Date']-combine_set['Camp_Start_Date']
combine_set['Diff_in_CampStart_Registration']=combine_set['Camp_Start_Date']-combine_set['Registration_Date']
combine_set['Diff_in_CampSEnd_Registration']=combine_set['Camp_End_Date']-combine_set['Registration_Date']
combine_set['Diff_in_Registration_Interaction']=combine_set['Registration_Date']-combine_set['First_Interaction']
combine_set['Diff_in_CampStart_First_Interaction']=combine_set['Camp_Start_Date']-combine_set['First_Interaction']


In [ ]:

drop_cols=['Employer_Category','Patient_ID','Health_Camp_ID','Camp_Start_Date','Camp_End_Date','Registration_Date',
           'Var4', 'Online_Follower','Facebook_Shared','Twitter_Shared','LinkedIn_Shared', 'Outcome']

train_df=combine_set[combine_set['Outcome'].isnull()==False]
test_df=combine_set[combine_set['Outcome'].isnull()==True]

train_df.to_csv('C:/Users/UF145GV/Documents/DL/testing/Healthcare Analytics/train_temp.csv', index=False)

train_df = train_df.sort_values(['Camp_Start_Date', 'Camp_End_Date', 'Patient_ID']).reset_index(drop=True)
test_df = test_df.sort_values(['Camp_Start_Date', 'Camp_End_Date', 'Patient_ID']).reset_index(drop=True)

X=train_df.drop(drop_cols, axis=1)
Y=train_df['Outcome']

X_main_test=test_df.drop(drop_cols,axis=1)

In [ ]:
X.head(10)

In [ ]:
# lg=LGBMClassifier(boosting_type='gbdt', max_depth=12, learning_rate=0.09, objective='binary', reg_alpha=0,
#                   reg_lambda=1, n_jobs=-1, random_state=100, n_estimators=500)

# lg.fit(X,Y)

# print(roc_auc_score(Y_test,lg.predict(X_test)))

In [ ]:


xg=XGBClassifier(booster='gbtree', max_depth=6, learning_rate=0.07, min_child_weight=5, subsample=0.8,
                 objective='binary:logistic', n_jobs=-1, colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.7,
                 random_state=100)

xg.fit(X,Y)

# print(roc_auc_score(Y_test,xg.predict(X_test)))

In [ ]:
#Checking For Permutation Importance
perm = PermutationImportance(xg,random_state=100).fit(X_test, Y_test)
eli5.show_weights(perm,feature_names=X_test.columns.tolist())

In [ ]:
#Predictions on Test Data
predictions=xg.predict_proba(X_main_test)
main_preds=[]
for i in range(0, len(X_main_test)):
    main_preds.append(predictions[i][1])



In [ ]:
#Final Submission File
d=pd.DataFrame()

d=pd.concat([d,test_df['Patient_ID'],test_df['Health_Camp_ID'], pd.DataFrame(main_preds,columns=['Outcome'])], axis=1)
d.head(10)

d.to_csv('C:/Users/UF145GV/Documents/DL/testing/Healthcare Analytics/main_test.csv', index=False)

In [ ]:
# submission_file=pd.read_csv('C:/Users/UF145GV/Documents/DL/testing/Healthcare Analytics/sample_submmission.csv')

# submission_file['Outcome']=main_preds

# submission_file.to_csv('C:/Users/UF145GV/Documents/DL/testing/Healthcare Analytics/main_test.csv', index=False)